In [95]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#追記
import json
import datetime
import math

from urllib import request
from bs4 import BeautifulSoup
import urllib.request
import requests
import re

from dateutil.relativedelta import relativedelta

import time

#importじゃないけど、個人的には省略されて良いことがあまりないので・・
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10000)

In [96]:
f = open("../data/drama_detail_0205_5907.json", 'r')
print(f)
drama_detail = json.load(f)

<_io.TextIOWrapper name='../data/drama_detail_0205_5907.json' mode='r' encoding='UTF-8'>


In [106]:
df_vr = pd.read_csv("../data/viewing_rating.csv", encoding='cp932')

In [107]:
df_vr

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,Episode8,Episode9,Episode10,Episode11,Episode12,Episode13,Episode14,Episode15,average,suii,start_at
0,CX,月21,シャーロック,12.8,9.3,9.9,10.6,9.3,8.3,9.9,8.9,9.9,8.8,9.8,-,-,-,-,9.90%,5-5-0,2019/10/1
1,CX,火22,まだ結婚できない男,11.5,7.7,10.0,9.5,10,8.9,8.5,8.6,9,9.7,-,-,-,-,-,9.40%,5-4-0,2019/10/1
2,TBS,火22,G線上のあなたと私,7.8,8.8,7.3,6.9,6.7,6.7,8.6,7.6,6.5,9.6,-,-,-,-,-,7.65%,2003/5/1,2019/10/1
3,NTV,水22,同期のサクラ,8.1,9.5,9.3,11.5,11.8,11.7,12.2,10.8,10.3,13.7,-,-,-,-,-,10.89%,5-4-0,2019/10/1
4,EX,木22,ドクターX,20.3,19.0,18.1,17.8,15.9,19.3,18.6,17.4,18.2,19.3,-,-,-,-,-,18.48%,3-6-0,2019/10/1
5,CX,木22,モトカレマニア,5.6,5.2,4.1,3.0,4.6,4.1,4.3,4.7,4.2,-,-,-,-,-,-,4.46%,3-5-0,2019/10/1
6,NTV,木23,CHEAT,4.2,3.0,4.4,3.0,3.1,2.9,3.5,3.7,3.5,3.2,-,-,-,-,-,3.45%,4-5-0,2019/10/1
7,TBS,金22,4分間のマリーゴールド,10.3,7.8,7.6,6.2,7.1,5.7,7.1,6.4,7,9.2,-,-,-,-,-,7.52%,4-5-0,2019/10/1
8,EX,金23,時効警察はじめました,7.7,6.1,5.9,4.9,6.3,6.2,5.5,5.5,-,-,-,-,-,-,-,6.01%,2001/5/1,2019/10/1
9,NTV,土22,俺の話は長い,8.4,9.0,8.9,8.4,8,7.7,7.4,9.1,7.2,10.4,-,-,-,-,-,8.45%,3-6-0,2019/10/1


In [111]:
# time型に
df_vr.start_at = pd.to_datetime(df_vr.start_at)

In [112]:
df_vr["drama_key"] = df_vr.start_at.dt.strftime("%y")+df_vr.start_at.dt.strftime("%m")+"_"+df_vr.TV_station+"_"+df_vr.time_table

0    シャーロック
Name: drama_title, dtype: object

In [123]:
df = pd.DataFrame(index = (), columns = ["drama_key","drama_title","sharp_title","sharp_num_cnt","sharp_num_epg",
                                        "start_time","sharp_detail_epg", ])


In [124]:
# df = df.append(pd.Series(["1100_ntv", "咲", "咲ます",1,2,21,"hogegeho"   ], index=df.columns),ignore_index = True)

In [125]:
cnt = 0
for drama_key in drama_detail.keys():
    print(drama_key)
    print(drama_detail[drama_key])
    for tmp_cnt in drama_detail[drama_key].keys():
#     df.iloc[cnt]["drama_key"] = drama_key
        df = df.append(pd.Series([drama_key,
                                  df_vr[df_vr.drama_key == drama_key].drama_title.values[0],
                                  drama_detail[drama_key][tmp_cnt]["sharp_title"],
                                  int(tmp_cnt) - 1,
                                  drama_detail[drama_key][tmp_cnt]["sharp_epg"],
                                  drama_detail[drama_key][tmp_cnt]["start_time"],
                                  drama_detail[drama_key][tmp_cnt]["program_detail"],
                                ], index=df.columns),ignore_index = True)
    
    cnt +=1; 
    if cnt>100 : break;
    


1910_CX_月21
{'2': {'sharp_title': '番組名シャーロック【危険な天才探偵×スマートな医師ー今夜、運命の出逢い】\u3000#01', 'sharp_epg': ['#01'], 'start_time': ['21', '22'], 'program_detail': '番組詳細【公式HP】  https://www.fujitv.co.jp/sherlock/  【公式Twitter】  https://twitter.com/sherlockcx  【公式Instagram】  https://www.instagram.com/SHERLOCK_cx/都内にある病院の中庭で、この病院に勤務する消化器内科医、赤羽栄光(あかばね・はるき/中尾明慶)が遺体となって発見された。捜査一課の警部・江藤礼二(佐々木蔵之介)は、何者かによって屋上から突き落とされたのではないかと病院関係者へ事情聴取するが、患者からの信頼も厚く、異性関係のトラブルとも皆無だった赤羽が殺されるはずなどないと口をそろえる。その様子を静かに聞いていたのは犯罪捜査専門コンサルタントの誉獅子雄(ディーン・フジオカ)である。公にはしていないが、江藤は、ずばぬけた観察眼と天才的な思考回路を持ち合わせる獅子雄の力を時折借りながら事件捜査に当たっている。彼らの証言に潜む虚偽を瞬時に見抜き、必要な情報と不要な情報を整理していく獅子雄。そんな中、事情調査中にも関わらずその場を静かに立ち去った1人の男がいた。この病院に勤務する精神科医の若宮潤一(岩田剛典)である。その様子を見逃さなかった獅子雄は、彼が何か真実を握っているのではないかと察知し、重要参考人として調査を開始する。ディーン・フジオカ\u3000  岩田剛典  \u3000/\u3000  佐々木蔵之介\u3000他【原作】  アーサー・コナン・ドイル『シャーロック・ホームズ』シリーズ\u3000  【脚本】  井上由美子(『昼顔～平日午後3時の恋人たち～』、『白い巨塔』)\u3000  【音楽】  菅野祐悟\u3000  【主題歌】  DEAN\u3000FUJIOKA『Shelly』(A-Sketch)\u3000  【プロデュース】  太田大(『モンテ・クリスト伯ー華麗なる復讐ー』、『レ・ミゼラブル\u3000終わりなき旅路

1910_TBS_金22
{'2': {'sharp_title': '番組名金曜ドラマ「4分間のマリーゴールド」♯1【血の繋がらない姉への秘めた想い】', 'sharp_epg': ['#302'], 'start_time': ['22', '23'], 'program_detail': '番組詳細手を重ねた人の「死の運命」が視える能力をもつ救急救命士の花巻みこと(福士蒼汰)。一度視た運命は必ず現実となり、救いたいのに救えないジレンマを抱えながら、それでも日々救急隊として全力を尽くしていた。みことにはある秘密があった。それはひとつ屋根に暮らす血の繋がらない姉・沙羅(菜々緒)を愛してしまったこと。その想いは日に日に募るばかりで…。そしてみことはある日、沙羅の悲しい運命を知ってしまい…。福士蒼汰、菜々緒、横浜流星、西村元貴、伊藤あさひ、関水渚、鈴木ゆうか、麻生祐未、三浦誠己、西尾まり、橋本じゅん、桐谷健太主題歌:平井堅「#302」(ソニー・ミュージックレーベルズ)  音楽:兼松衆\u3000櫻井美希脚本:櫻井剛  演出:河野圭太(共同テレビ)、城宝秀則(共同テレビ)\u3000プロデューサー:橋本芙美(共同テレビ)  ◇番組HP  http://www.tbs.co.jp/4mari/  ◇番組instagram  https://www.instagram.com/4mari_tbs/  ◇番組Twitter  https://twitter.com/4mari_tbs  @4mari_tbs  番組の内容と放送時間は変更になる可能性があります。'}, '3': {'sharp_title': '番組名金曜ドラマ「4分間のマリーゴールド」♯2【料理担当の末っ子・藍の抱える秘密】', 'sharp_epg': ['#302'], 'start_time': ['22', '22'], 'program_detail': '番組詳細「死の運命」が視える能力をもつ救命士みこと(福士蒼汰)は、愛する義姉・沙羅(菜々緒)が余命1年という運命を知り、絶対に沙羅を守ると心に誓う。まずは沙羅の健康に問題がないか探るみこと。その様子にいち早く気づく義弟・藍(横浜流星)だったが、実は藍にも抱えている秘密があった。そんな中、みことは救命の現場で和江(松金よね子)という高齢女性に出会う

1907_TBS_火22
{'2': {'sharp_title': '番組名火曜ドラマ「Heaven?～ご苦楽レストラン～」第1話', 'sharp_epg': None, 'start_time': ['22', '23'], 'program_detail': '番組詳細フレンチレストランで働いていた伊賀観(福士蒼汰)は周りからまったく評価されていなかったが、ある時来客した謎の女・黒須仮名子(石原さとみ)にはなぜか高評価。近日オープンの彼女の店「ロワン・ディシー」で働くことを決意する。早速、ミーティングのため彼女の店に訪れた伊賀を待っていたのは川合太一(志尊淳)、堤計太郎(勝村政信)、小澤幸應(段田安則)、山縣重臣(岸部一徳)。6人による究極の喜劇が今幕を開ける!石原さとみ(黒須仮名子)…みんなを振り回す風変わりなオーナー  福士蒼汰(伊賀観)…笑顔が苦手な真面目な青年  志尊淳(川合太一)…戦力にならない天然愛されキャラ  勝村政信(堤計太郎)…経営に悪戦苦闘する元牛丼屋店長  段田安則(小澤幸應)…秘密を抱える元三ツ星天才シェフ  岸部一徳(山縣重臣)…資格取得が趣味のお茶目な元銀行員佐々木倫子「Heaven?～ご苦楽レストラン～」(小学館ビッグスピリッツコミックス刊)  吉田恵里香主題歌:「真夏の夜の匂いがする」あいみょん(nuBORDER/Warner Music Japan)  音楽:井筒昭雄プロデュース:瀬戸口克陽  演出:木村ひさし、松木彩、村尾嘉昭◇番組HP  https://www.tbs.co.jp/Heaven_tbs/    ◇twitter  @heaven_tbs  https://twitter.com/heaven_tbs    ◇Instagram  kawai_iigram  番組の内容と放送時間は変更になる場合があります。'}, '3': {'sharp_title': '番組名火曜ドラマ「Heaven?～ご苦楽レストラン～」シェフの秘密が明らかに!第2話', 'sharp_epg': None, 'start_time': ['22', '23'], 'program_detail': '番組詳細仮名子(石原さとみ)は、店がはやらないのはシェフの小澤の料理に問題があると主張する。実は小澤は誰にも言えない秘密を抱えているの

1907_TBS_金22
{'2': {'sharp_title': '番組名金曜ドラマ「凪のお暇」(なぎのおいとま)\u3000第01話', 'sharp_epg': None, 'start_time': ['22', '23'], 'program_detail': '番組詳細家電メーカーの営業事務として働く大島凪(なぎ)はいわゆる「空気を読みすぎる」アラサー女子。上司や同僚、SNS、友達、そして、彼氏・・・ありとあらゆる空気を読みまくっていた。本人ですら「なんだかなぁ」と思ってしまう毎日だったが、あることを機に、仕事も辞めスマホも解約、彼氏と連絡を絶ち、毎朝1時間も費やしたヘアセットもやめ、ひどいくせ毛の天然パーマもそのままに「人生リセット」を図るー。「人生リセット」して引っ越した先は、6畳一間のクーラーなしのボロアパート。危険な匂いのする隣人・ゴンをはじめとする個性豊かな住人たちとの出会い。そこに別れたはずの彼氏、慎二がやってきて・・・。黒木華  高橋一生  中村倫也  市川実日子    瀧内公美  大塚千弘  藤本泉  水谷果穂  唐田えりか  白鳥玉季  中田クルミ  谷恭輔  田本清嵐  モクタール  松永拓野  ファーストサマーウイカ    \u3000\u3000\u3000〇  片平なぎさ  武田真治  \u3000\u3000\u3000〇  吉田羊    三田佳子音楽:パスカルズ    主題歌:miwa「リブート」  原作:コナリミサト(秋田書店「エレガンスイブ」連載中)  脚本:大島里美  プロデュース:中井芳彦  演出:坪井敏雄  製作著作:TBS  ◇番組HP  https://www.tbs.co.jp/NAGI_NO_OITOMA/    ◇twitter  @nagino_oitoma  https://twitter.com/nagino_oitoma  番組の内容と放送時間は変更になる可能性があります。'}, '3': {'sharp_title': '番組名金曜ドラマ「凪のお暇」(なぎのおいとま)\u3000第02話', 'sharp_epg': None, 'start_time': ['22', '22'], 'program_detail': '番組詳細人生リセットするために全てを捨てて東京郊外のアパートに引っ越した

1904_CX_月21
{'2': {'sharp_title': '番組名ラジエーションハウス～放射線科の診断レポート～【新たな医療ドラマ】\u3000#01', 'sharp_epg': ['#01'], 'start_time': ['21', '22'], 'program_detail': '番組詳細【ホームページ】  https://www.fujitv.co.jp/radiationhouse/\u3000  【Twitter】  https://twitter.com/radiation_2019\u3000  公式LINEアカウントも登録してください。\u3000五十嵐唯織(窪田正孝)は“写真には必ず真実が映る"と信じる放射線技師。アメリカで最も権威ある放射線科医から才能を認められた唯織は、帰国後ずっと想いを寄せる幼なじみの甘春杏(本田翼)が放射線科医として勤務する甘春総合病院に採用される。  \u3000初出勤の日、唯織はバスで甘春総合病院へと向かう。バスには、唯織と共に採用された新人放射線技師の広瀬裕乃(広瀬アリス)の姿もあった。その途中、バス運転手・天野(春海四方)の異変に気づいた唯織は、駆けつけた救急隊員に血管造影ができる病院への搬送を指示する。  \u3000天野は甘春総合病院に搬送される。受け入れたのは救急担当の整形外科医・辻村駿太郎(鈴木伸之)。だが天野は、自力で立てるほど元気だった。  \u3000放射線科は、診療部長兼放射線科長の鏑木安富(浅野和之)のもと、ギャンブル好きの小野寺俊夫(遠藤憲一)、男勝りな独身女子・黒羽たまき(山口紗弥加)らが放射線技師として働いている。彼らの間では、唯織が病院長・大森渚(和久井映見)の隠し子ではないか、というウワサが…。  \u3000唯織は、バスの一件で遅刻しながらも杏と再会を果たす。だが杏は、唯織のことなどまったく覚えていなかった。  \u3000そんな折、甘春総合病院に世界的な写真家・菊島亨(イッセー尾形)が搬送される。頭痛を訴える菊島の頭部MRI検査を行う小野寺たち。しかし、画像の左上が黒く抜け欠損していて…。窪田正孝\u3000  本田翼\u3000    広瀬アリス\u3000  浜野謙太\u3000  丸山智己\u3000  矢野聖人\u3000    山口紗弥加  \u3000・\

...

1804_TBS_火22
{'2': {'sharp_title': '番組名火曜ドラマ「花のち晴れ～花男\u3000Next\u3000Season～」(1)★花男新章開幕!', 'sharp_epg': None, 'start_time': ['22', '23'], 'program_detail': '番組詳細伝説の4人組“F4"が卒業して10年、名門・英徳学園はかつての輝きを失いかけていた。高等部2年生・江戸川音(杉咲花)は、父の会社が倒産したため、母・由紀恵(菊池桃子)と二人で貧乏生活を送りながら、“隠れ庶民"として学園に通っていた。そこでカリスマとして崇められているのが、神楽木晴(平野紫耀)。父・巌(滝藤賢一)は、道明寺財閥と肩を並べる日本三大財閥の1つである神楽木ホールディングスの若き会長だ。道明寺に憧れる晴は学園に栄光を取り戻すため、正しき5人“C5(シーファイブ)"を結成。高等部に入ってからは、寄付金未納・学費滞納の生徒を退学させる“庶民狩り"を行っていた。その庶民狩りにおびえる音。実は彼女には、馳天馬(中川大志)という婚約者がおり、18歳まで学園にいる事が結婚の条件だった…。隠れ庶民の音と庶民狩りをしているC5の晴が最悪の出会いを果たし、物語は衝撃の展開を迎える…!杉咲花  平野紫耀(King & Prince)  中川大志  濱田龍臣  飯豊まりえ  今田美桜  鈴木仁  中田圭祐    翔\u3000  ○  反町隆史    松本潤(友情出演)  ○  木南晴夏  堀内敬子  テット・ワダ  志賀廣太郎  高岡早紀  滝藤賢一  菊池桃子神尾葉子  「花のち晴れ～花男 Next Season～」  (集英社「少年ジャンプ+」連載)  吉田恵里香  (「センセイ君主」「ヒロイン失格」「レオン」ほか)  【主題歌】  King & Prince  「シンデレラガール」  (Johnnys\' Universe)    【イメージソング】  宇多田ヒカル  (EPICレコードジャパン)    【音楽】  大間々昂【プロデューサー】  瀬戸口克陽  (花男シリーズ、99.9ほか)    【協力プロデューサー】  齊藤彩奈    【演出】  石井康晴  (花男シリーズ、ウロボロスほか)◇番組HP http://www.tbs.co.jp/hana_

1804_EX_金23
{'2': {'sharp_title': '番組名家政夫のミタゾノ\u3000#1', 'sharp_epg': ['#1'], 'start_time': ['23', '00'], 'program_detail': '番組詳細家政夫の三田園薫(松岡昌宏)は、勤め先の家政婦紹介所所長・結頼子(余貴美子)から、新人家政婦の五味麻琴(剛力彩芽)とともに、小さな町工場に派遣される。従業員らを家族同然に思う社長の花沢進助(橋本じゅん)や、彼らを献身的に支える妻(森脇英理子)と接したミタゾノは「油の汚れの下に素顔を隠した人もいたりして…」と意味深な言葉を残す。その言葉通り、花沢夫妻の知られざる顔がミタゾノの手により次々と暴かれ…!?松岡昌宏、剛力彩芽、椿鬼奴、内藤理沙、余貴美子  【ゲスト】橋本じゅん、森脇英理子、ジェシー八津弘幸七高剛島茂子『戯言』(ジェイ・ストーム)【ゼネラルプロデューサー】内山聖子(テレビ朝日)  【プロデューサー】秋山貴人(テレビ朝日)、木曽貴美子(MMJ)、椋尾由希子(MMJ)☆番組HP  \u3000http://www.tv-asahi.co.jp/mitazono/  ☆Twitter  \u3000https://twitter.com/mitazono_desu'}, '3': {'sharp_title': '番組名家政夫のミタゾノ\u3000#2', 'sharp_epg': ['#2'], 'start_time': ['23', '00'], 'program_detail': '番組詳細三田園薫(松岡昌宏)と五味麻琴(剛力彩芽)が派遣先を訪れると、家主の矢吹研一(近藤公園)が若い女性といる不倫現場に遭遇する…!研一は、妻・理沙子(矢田亜希子)が依頼した家政婦たちの登場に驚きつつ、酒にのまれ昨夜のことは覚えていないと弁明。麻琴はその態度に怒り心頭。一方ミタゾノは、不倫を隠すための家事テクを披露!そこに理沙子が帰宅し研一の行動を怪しむが、三田園はむしろ理沙子に疑問を抱き…!?松岡昌宏、剛力彩芽、椿鬼奴、内藤理沙、余貴美子  【ゲスト】矢田亜希子、近藤公園、迫田孝也、片山萌美小峯裕之片山修(テレビ朝日)島茂子『戯言』(ジェイ・ストーム)【ゼネラルプロデューサー】内山聖子(テレビ朝日)  【プロデューサー】秋

1801_CX_月21
{'2': {'sharp_title': '番組名海月姫【東村アキコの傑作をドラマ化!月9史上最も奇妙な三角関係!】\u3000#01', 'sharp_epg': ['#01'], 'start_time': ['21', '22'], 'program_detail': '番組詳細【公式HP】http://www.fujitv.co.jp/kuragehime/クラゲを愛しすぎてしまった筋金入りの“クラゲオタク女子"が、とある兄弟と三角関係になり、自分には一生縁はないと思っていた恋を知り、新しい自分、新しい生き方を見つけていく姿を、ギャグを交えて描きます。  ある朝、『天水館』自室のベッドで目覚めた倉下月海(芳根京子)は、隣を見てビックリ。なんと上半身裸の男が寝ているではないか…月海は昨晩のことを思い起こす。月海は、久しぶりに近所の熱帯魚店にクララと名前までつけているクラゲを見に行った。月海は、その水槽にタコクラゲと一緒に飼ってはいけないミズクラゲが一緒に入っていることに気づいた。注意しようと思う月海だが、相手はオシャレな男性店員。月海が大苦手の人種だ。月海は勇気を振り絞って注意するのだが、追い出され、店の前で倒れてしまう。そんな店員を注意するスタイル抜群の美女が月海を助けてくれた。月海がわけを話すと、その美女はクララを買ってしまおうとお持ち帰り。美女は月海とともに『天水館』まで来ると部屋まで見たいと言い出した。美女は月海の部屋に勝手に泊まることにしてしまい…。  そして、次の朝、目覚めた月海が大声を出すことに。美女が上半身裸の男、鯉淵蔵之介(瀬戸康史)に変身していたのだ。蔵之介によると女装は趣味らしい。慌てる月海は『天水館』は男子禁制の尼寺のようなところだと蔵之介に説明。女装姿に戻った蔵之介が月海と部屋を出ると、独特な住人たちがいて…。芳根京子\u3000  瀬戸康史\u3000  工藤阿須加\u3000  木南晴夏\u3000  松井玲奈\u3000  内田理央\u3000  富山えり子  \u3000・\u3000  泉里香\u3000  安井順平\u3000  要潤  \u3000・\u3000  床嶋佳子\u3000  北大路欣也\u3000    他【原作】  東村アキコ「海月姫」(講談社「Kiss」所載)\u3000   

1801_CX_木22
{'2': {'sharp_title': '番組名<木曜劇場>隣の家族は青く見える\u3000#01', 'sharp_epg': ['#01'], 'start_time': ['22', '23'], 'program_detail': "番組詳細スキューバダイビングのインストラクターをしている五十嵐奈々(深田恭子)とおもちゃメーカーに勤務する大器(松山ケンイチ)夫妻は、小さなアパートで暮らしながら資金を貯め、コーポラティブハウスを購入する。同じコーポラティブハウスに住むのは、結婚を控えたバツイチのスタイリスト・川村亮司(平山浩行)とネイリストの杉崎ちひろ(高橋メアリージュン)のカップル、2人の娘を持つ商社マンの小宮山真一郎(野間口徹)と深雪(真飛聖)夫妻、そしてこのコーポラティブハウスの設計も手掛けた建築士の広瀬渉(眞島秀和)。  子どもが大好きな奈々と大器は、1年ほど前から子作りを始めていた。そんな矢先、大器の妹・琴音(伊藤沙莉)が、家業の焼き鳥店で働く糸川啓太(前原滉)の子を妊娠。夫の健作(春海四方)と店を切り盛りする大器たちの母・聡子(高畑淳子)は大喜びし、次は奈々の番だと期待を寄せる。  奈々たちが帰宅すると、共有スペースでは小宮山一家が餅つきをしていた。さっそく手伝い始める奈々たち。その際、奈々は、渉を訪ねてきた青木朔(北村匠海)と出会う。実は朔は渉の恋人で、渉と一緒に暮らすつもりでいた。  早く子どもが欲しいと願う奈々は、大器を説得し、不妊治療専門のクリニックを訪れる。そこで奈々たちは、1年以上避妊していないにもかかわらず妊娠に至らないのは不妊症と言える、という片岡医師(伊藤かずえ)の言葉にショックを受け…。深田恭子\u3000  松山ケンイチ\u3000    平山浩行\u3000  高橋メアリージュン\u3000  北村匠海\u3000  眞島秀和\u3000  真飛聖\u3000  野間口徹  \u3000・\u3000  須賀健太\u3000  伊藤沙莉\u3000  前原滉\u3000  寿大聡  \u3000・\u3000  春海四方\u3000  伊藤かずえ\u3000  高畑淳子【脚本】  中谷まゆみ(『ラスト・シンデレラ』、『ディア・シスター』、『ウォーターボーイズ』他)\u3000    【プロデュース

1801_TBS_日21
{'2': {'sharp_title': '番組名日曜劇場「99.9-刑事専門弁護士-SEASONII」♯1「元裁判官の依頼」', 'sharp_epg': None, 'start_time': ['21', '22'], 'program_detail': '番組詳細刑事事件専門ルームでは室長が定着せず、佐田(香川照之)は斑目(岸部一徳)から、民事の兼務と見つかったらマネージングパートナーを譲るという条件のもと渋々承諾。深山(松本潤)と佐田の名コンビが復活する。  そんな中、とある殺人事件の弁護依頼が舞い込む。依頼人は、鈴木二郎の娘・加代。  鈴木が借金をしていた沢村ファイナンスの社長  が殺害され、鈴木はその容疑者として逮捕・起訴された。そして、加代の付き添いで来ていたのが、尾崎舞子(木村文乃)だった。  舞子はエリート裁判官だったが、あることが理由で退官していた。  鈴木の無実を証明しようとする深山たちに対して、舞子は調書を見る限り鈴木の犯行で間違いないから、罪を認めて情状酌量を求めるべきだと主張。  深山と対立する。一方、佐田はその様子を見てあることを画策する。深山大翔・・・松本潤  佐田篤弘・・・香川照之  尾崎舞子・・・木村文乃  明石達也・・・片桐仁  藤野宏樹・・・マギー  中塚美麗・・・馬場園梓  落合陽平・・・馬場徹  佐田由紀子・・・映美くらら坂東健太・・・池田貴史  加奈子・・・岸井ゆきの    川上憲一郎・・・笑福亭鶴瓶  斑目春彦・・・岸部一徳(1話出演)  志賀誠・・・藤本隆宏  戸川奈津子・・・渡辺真起子  阿部充・・・長塚圭史  鈴木加代・・・谷村美月  鈴木二郎・・・半海一晃  伊藤亜紀・・・新妻聖子脚本・・・宇田学  トリック監修・・・蒔田光治  音楽・・・井筒昭雄  主題歌・・・「Find The Answer」嵐  \u3000\u3000\u3000\u3000\u3000\u3000(ジェイ・ストーム)  プロデュース・・・瀬戸口克陽  \u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000東仲恵吾  演出・・・木村ひさし◇番組HP  http://www.tbs.co.jp/999tbs/  ◇twitter  @999_2018tbs

In [126]:
df

,drama_key,drama_title,sharp_title,sharp_num_cnt,sharp_num_epg,start_time,sharp_detail_epg
0,1910_CX_月21,シャーロック,番組名シャーロック【危険な天才探偵×スマートな医師ー今夜、運命の出逢い】 #01,1,[#01],"[21, 22]",番組詳細【公式HP】 https://www.fujitv.co.jp/sherlock/...
1,1910_CX_月21,シャーロック,番組名シャーロック【探偵×医師最強バディ始動!天才VS聖女の謎解き心理戦】 #02,2,[#02],"[21, 22]",番組詳細「FIVBワールドカップバレーボール2019 男子 日本×ブラジル」延長の際、放送時...
2,1910_CX_月21,シャーロック,番組名シャーロック【天才VS詐欺師の騙し合い!ノンストップのどんでん返し】 #03,3,[#03],"[21, 21]",番組詳細【公式HP】 https://www.fujitv.co.jp/sherlock/...
3,1910_CX_月21,シャーロック,番組名シャーロック【試合直前に消えたボクシング世界王者!オレンジの傘の謎】 #04,4,[#04],"[21, 21]",番組詳細【公式HP】 https://www.fujitv.co.jp/sherlock/...
4,1910_CX_月21,シャーロック,番組名シャーロック【歩く死体の謎!熱帯魚と母の愛…その真相、正義か狂気か】 #05,5,[#05],"[21, 21]",番組詳細【公式HP】 https://www.fujitv.co.jp/sherlock/...
5,1910_CX_月21,シャーロック,番組名シャーロック【前世で犯した殺人?カナリアとビデオテープが語る真相…】 #06,6,[#06],"[21, 21]",番組詳細【公式HP】 https://www.fujitv.co.jp/sherlock/...
6,1910_CX_月21,シャーロック,番組名シャーロック【少年シャーロック現る!消えた祖父と開かずの金庫の謎!】 #07,7,[#07],"[21, 21]",番組詳細【公式HP】 https://www.fujitv.co.jp/sherlock/...
7,1910_CX_月21,シャーロック,番組名シャーロック【殺しの暗号!心中事件現場に残されたライオンの写真の謎】 #08,8,[#08],"[21, 21]",番組詳細【公式HP】 https://www.fujitv.co.jp/sherlock/...
8,1910_CX_月21,シャーロック,番組名シャーロック【最終章へ…別れの予感!最後の晩餐は密室殺人レストラン】 #09,9,[#09],"[21, 21]",番組詳細【公式HP】 https://www.fujitv.co.jp/sherlock/...
9,1910_CX_月21,シャーロック,番組名シャーロック【最終回直前!あの男がついに覚醒…ラスト3分世界が覆る】 #10,10,[#10],"[21, 21]",番組詳細【公式HP】 https://www.fujitv.co.jp/sherlock/...
